In [1]:
import datetime
import os
import sys
import pandas as pd
import models.train_model as tm
import models.predict_model as pm
import features.build_features as build_features
from utilities.path_utils import *
from utilities.load_utils import *
from utilities.constants import * 


In [12]:
other_path = get_others_path("netrisk_casco")
eurotax = pd.read_csv(other_path / "full_trim_list.csv", low_memory=False)
prices = pd.read_csv(other_path / "price.csv", low_memory=False)[['eurotax_code', 'new_price_1_gross']]

In [16]:
prices

,eurotax_code,new_price_1_gross
0,0101B001,5450000.0
1,0101B002,5780000.0
2,0101B003,6350000.0
3,0101B004,6680000.0
4,0101B005,7200000.0
...,...,...
123992,28300150,NaN
123993,28300151,NaN
123994,28300153,NaN
123995,78300031,NaN


In [19]:
netrisk_cars

,Unnamed: 0,eurotax_code,car_make_year,car_make_year_code,car_make,car_make_code,car_model,car_model_code,car_trim_id,car_trim_name,car_full_name,kw,ccm,kg,car_value
12,12,0279B005,2022,2,ALFA ROMEO,3,Giulia,2,4,Giulia 2.0 Veloce Q4 Aut.,ALFA ROMEO_Giulia_2.0 Veloce Q4 Aut.,206,1995,1530,14397000.0
82,82,0207B010,2019,5,ALFA ROMEO,3,MiTo,5,4,MiTo 1.4 TB Multiair Veloce TCT (5 sz.),ALFA ROMEO_MiTo_1.4 TB Multiair Veloce TCT (5 ...,125,1368,1245,7094000.0
97,97,0279B006,2018,6,ALFA ROMEO,3,Giulia,3,4,Giulia 2.0 Elegante Aut.,ALFA ROMEO_Giulia_2.0 Elegante Aut.,147,1995,1504,9990000.0
99,99,0209B007,2018,6,ALFA ROMEO,3,Giulietta,4,4,Giulietta 1.4 TB MultiAir Sport,ALFA ROMEO_Giulietta_1.4 TB MultiAir Sport,110,1368,1290,7555000.0
115,115,0207B007,2018,6,ALFA ROMEO,3,MiTo,5,4,MiTo 1.4 MPI MiTo (5 szem.),ALFA ROMEO_MiTo_1.4 MPI MiTo (5 szem.),57,1368,1155,4740000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40740,40740,9059B012,2014,10,VOLKSWAGEN,132,CC,2,4,CC 1.4 TSI [5 személy],VOLKSWAGEN_CC_1.4 TSI [5 személy],118,1390,1430,9710650.0
40771,40771,9008B023,2014,10,VOLKSWAGEN,132,Golf Plus,4,4,Golf VI Plus 1.2 TSI Highline DSG,VOLKSWAGEN_Golf VI Plus_1.2 TSI Highline DSG,77,1197,1325,6381980.0
40805,40805,9015B004,2014,10,VOLKSWAGEN,132,New Beetle,7,4,Beetle 1.2 TSI Design DSG,VOLKSWAGEN_Beetle_1.2 TSI Design DSG,77,1197,1222,5778730.0
40861,40861,9006B004,2014,10,VOLKSWAGEN,132,Scirocco,11,4,Scirocco 2.0 TFSI,VOLKSWAGEN_Scirocco_2.0 TFSI,155,1984,1298,8733000.0


In [20]:
postal_codes = pd.read_csv(get_others_path("netrisk_casco") / "hungary_postal_codes.csv")['postal_code']
population = pd.read_csv(get_others_path("netrisk_casco") / "hungary_postal_codes_population.csv").rename(
    columns={'PostalCode': 'postal_code',
             'Population': 'population'})[['postal_code', 'population']]


def to_int(x):
    try:
        return int(str(x).replace(',', ''))
    except Exception as e:
        return None


population['population'] = population['population'].apply(to_int)
population = population.dropna()


In [19]:

base_profile = read_data_frame(get_profiles_for_crawling_transposed("netrisk_casco_sampled_data"))
base_profile[(base_profile['CarModel'] == 'Corolla')]
base_profile = base_profile.iloc[78]
base_profile

Unnamed: 0                  78
Age                         77
isRecent                  True
LicenseAge                  18
PostalCode                1039
BonusMalus                 B10
BonusMalusCode               1
kw                          73
CarMake                 TOYOTA
CarAge                       7
CarModel               Corolla
CarModelSpecific             5
ccm                       1329
kg                        1225
car_value              12558.0
postal_code               1039
Latitude               47.4984
Longitude              19.0404
Category                   1.0
PostalCode2                 10
PostalCode3                103
CarMakerCategory             1
PolicyStartDate     2023_01_01
Name: 78, dtype: object

In [20]:

base_profile['Age'] = 35
base_profile['CarAge'] = 1
base_profile['BonusMalus'] = 'B10'
base_profile['BonusMalusCode'] = 1
base_profile['PostalCode'] = 1011
base_profile['CarModelSpecific'] = 1
base_profile.to_csv(get_incremental_base_profile_path("netrisk_casco", "v1"))

In [22]:
postal_codes

0       7188
1       7300
2       7304
3       7305
4       7331
        ... 
3566    8997
3567    8998
3568    8998
3569    8999
3570    8999
Name: postal_code, Length: 3571, dtype: int64

In [44]:
other_path = get_others_path("netrisk_casco")
netrisk_cars = pd.read_csv(other_path / "netrisk_cars.csv", low_memory=False)

In [45]:
netrisk_cars = netrisk_cars.drop_duplicates('eurotax_code')
netrisk_cars = netrisk_cars[netrisk_cars['car_make_year'] >= 2014]
netrisk_cars['car_value'] = netrisk_cars['car_value'] * FORINT_TO_EUR

In [46]:
values = pd.DataFrame(columns=['feature', 'min', 'max', 'step', 'values'])
features_to_vary = pd.Series([('CarAge', 'CarMake', 'CarModel', 'CarModelSpecific')])
car_values = netrisk_cars[['car_make_year', 'car_make', 'car_model', 'car_trim_id']].values
car_values[:, 0] = datetime.today().year - car_values[:, 0]
car_values = list(map(tuple, car_values))

feature_values = pd.Series([car_values])
values['values'] = feature_values
values['feature'] = features_to_vary
values['min'] = pd.Series([None, 19, 0, None])
values['min'] = pd.Series([None])
values['max'] = pd.Series([None, 79, 10, None])
values['max'] = pd.Series([None])
values['step'] = pd.Series([None, 2, 1, None])
values['step'] = pd.Series([None])
values.to_csv(get_incremental_values_path("netrisk_casco", "all_cars_for_crawling"))

In [40]:
netrisk_cars

,Unnamed: 0,eurotax_code,car_make_year,car_make_year_code,car_make,car_make_code,car_model,car_model_code,car_trim_id,car_trim_name,car_full_name,kw,ccm,kg,car_value
0,0,0280B012,2022,2,ALFA ROMEO,3,Giulia,2,1,Giulia 2.0 Sprint Aut.,ALFA ROMEO_Giulia_2.0 Sprint Aut.,148,1995,1429,17790000.0
1,1,0279B003,2022,2,ALFA ROMEO,3,Giulia,2,2,Giulia 2.0 Super Aut.,ALFA ROMEO_Giulia_2.0 Super Aut.,148,1995,1429,10991000.0
11,11,0280B013,2022,2,ALFA ROMEO,3,Giulia,2,3,Giulia 2.0 TI Q4 Aut.,ALFA ROMEO_Giulia_2.0 TI Q4 Aut.,206,1995,1530,22010000.0
12,12,0279B005,2022,2,ALFA ROMEO,3,Giulia,2,4,Giulia 2.0 Veloce Q4 Aut.,ALFA ROMEO_Giulia_2.0 Veloce Q4 Aut.,206,1995,1530,14397000.0
22,22,0279B004,2022,2,ALFA ROMEO,3,Giulia,2,5,Giulia 2.9 T V6 Quadrifoglio Aut.,ALFA ROMEO_Giulia_2.9 T V6 Quadrifoglio Aut.,375,2891,1695,24309000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40931,40931,9057B012,2014,10,VOLKSWAGEN,132,Touran,15,7,Touran 1.2 TSI Trendline BlueMotion [7sz],VOLKSWAGEN_Touran_1.2 TSI Trendline BlueMotion...,77,1197,1385,6469610.0
40936,40936,9057B011,2014,10,VOLKSWAGEN,132,Touran,15,8,Touran 1.2 TSI Trendline [7sz],VOLKSWAGEN_Touran_1.2 TSI Trendline [7sz],77,1197,1378,6404840.0
40941,40941,9057B006,2014,10,VOLKSWAGEN,132,Touran,15,9,Touran 1.4 TSI Comfortline,VOLKSWAGEN_Touran_1.4 TSI Comfortline,103,1390,1441,6634710.0
40946,40946,9020B005,2014,10,VOLKSWAGEN,132,Up!,16,7,Up! 1.0 Move Up! ASG,VOLKSWAGEN_Up!_1.0 Move Up! ASG,55,999,848,3150150.0


In [49]:
pc = (pd.read_csv(get_others_path('punkta') / 'poland_postal_codes.csv').drop_duplicates(subset = ['postal_code']).to_csv(get_others_path('punkta') / 'poland_postal_codes.csv', sep = ','))

In [41]:

values = pd.DataFrame(columns=['feature', 'min', 'max', 'step', 'values'])
features_to_vary = pd.Series([('CarAge', 'CarMake', 'CarModel', 'kw', 'ccm', 'kg', 'car_value')])
car_values = netrisk_cars[['car_make_year', 'car_make', 'car_model', 'kw', 'ccm', 'kg', 'car_value']].values
car_values[:, 0] = datetime.today().year - car_values[:, 0]
car_values = list(map(tuple, car_values))

feature_values = pd.Series([car_values])
values['values'] = feature_values
values['feature'] = features_to_vary
values['min'] = pd.Series([None, 19, 0, None])
values['min'] = pd.Series([None])
values['max'] = pd.Series([None, 79, 10, None])
values['max'] = pd.Series([None])
values['step'] = pd.Series([None, 2, 1, None])
values['step'] = pd.Series([None])
values.to_csv(get_incremental_values_path("netrisk_casco", "all_cars_for_scoring"))

In [43]:
 
population = population[population['population'].between(0, 1000000)]

postal_codes_list = [postal_codes.to_list()]
values = pd.DataFrame(columns=['feature', 'min', 'max', 'step', 'values'])
features_to_vary = pd.Series(['PostalCode'])
feature_values = pd.Series(postal_codes_list)
values['values'] = feature_values
values['feature'] = features_to_vary
values['min'] = pd.Series([None])
values['min'] = pd.Series([None])
values['max'] = pd.Series([None])
values['max'] = pd.Series([None])
values['step'] = pd.Series([None])
values['step'] = pd.Series([None])
values.to_csv(get_incremental_values_path("netrisk_casco", "all_postal_codes"))